In [ ]:
from tinysim.topdown_driving.widget import TopDownDrivingWidget

widget = TopDownDrivingWidget()
widget.render()

In [ ]:
from tinysim.topdown_driving.tk import TopDownDrivingTkFrontend
from tinysim.topdown_driving import TopDownDrivingEnv
import numpy as np

under = TopDownDrivingEnv(num_envs=2)
env = TopDownDrivingTkFrontend(sim_env=under)
env.show_rays = True

env.render()
car_index = 0

for _ in range(500):
    throttle = np.zeros(2, dtype=np.float32)
    steer = np.zeros(2, dtype=np.float32)
    if "Up" in env.keys:
        throttle[car_index] = 1.0
    if "Down" in env.keys:
        throttle[car_index] = -1.0
    if "Left" in env.keys:
        steer[car_index] = -1.0
    if "Right" in env.keys:
        steer[car_index] = 1.0
    if "Shift_L" in env.keys:
        car_index = (car_index + 1) % env.sim_env.num_envs

    action = {"throttle": throttle, "steer": steer}
    state = await env.step(action, dt=0.02)
    # print(state["reward"])

### Example Neural Network for controlling the car

Input: `state["rays"]`, 5 hit rays by distance, `[d0, d1, d2, d3, d4]`

Output layer with 2 neurons `[throttle, steer]`

In [ ]:
from tinysim.topdown_driving.tk import TopDownDrivingTkFrontend
from tinysim.topdown_driving import TopDownDrivingEnv
import numpy as np


class SmallNN:
    def __init__(self, in_dim=5, hidden=8, out_dim=2):
        self.w1 = np.random.randn(in_dim, hidden) * 0.5
        self.b1 = np.zeros(hidden)
        self.w2 = np.random.randn(hidden, out_dim) * 0.5
        self.b2 = np.zeros(out_dim)

    def forward(self, x):
        h = np.tanh(x @ self.w1 + self.b1)
        return np.tanh(h @ self.w2 + self.b2)


under = TopDownDrivingEnv(num_envs=25)
nets = [SmallNN() for _ in range(under.num_envs)]

env = TopDownDrivingTkFrontend(sim_env=under)
env.render()

fitness = np.zeros(under.num_envs, dtype=np.float32)
action = {"throttle": 0.0, "steer": 0.0}
state = await env.step(action, dt=0.00)

for _ in range(600):
    throttle = np.zeros(under.num_envs, dtype=np.float32)
    steer = np.zeros(under.num_envs, dtype=np.float32)

    for i in range(under.num_envs):
        rays = state["rays"][i]
        output = nets[i].forward(rays)
        throttle[i] = output[0]
        steer[i] = output[1]

    action = {"throttle": throttle, "steer": steer}
    state = await env.step(action, dt=0.02)

    fitness += state["reward"]

print("Fitness:", fitness)